In [38]:
import json, os

import pandas as pd
import numpy as np

from konlpy.tag import Okt
import nltk

from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model
from keras.layers import Dense 
from keras import optimizers, losses, metrics

In [39]:
def read_csv():
    # pd로 csv 읽기
    return pd.concat([pd.read_csv("data/neg_opinion12.csv"), pd.read_csv("data/pos_opinion12.csv"),\
                      pd.read_csv("data/neg_opinion19.csv"), pd.read_csv("data/pos_opinion19.csv")])\
            .reset_index().drop('index', axis=1) # 인덱스 초기화

In [40]:
def check_pos():
    # 이미 형태소 전처리 된 json 파일 있는지 확인
    fname = "pos_tag.json"
    if os.path.isfile(fname):
        print("// 형태소 전처리 파일 존재")
        with open(fname, encoding="utf-8-sig") as f:
            data = pd.DataFrame(json.load(f), columns=["comment", "risk"])
    else: 
        print("// 형태소 전처리 파일 미존재")
        # okt로 형태소 분석하기
        data = read_csv()
        okt = Okt()
        def tokenize(doc):
            return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]
            # norm은 정규화(오타 및 띄어쓰기 교정), stem은 근어화
        data['comment'] = data['comment'].apply(tokenize)
        print("// 형태소 전처리 진행")
        pos_tag = [(data['comment'][i], int(data['risk'][i])) for i in range(len(data))]
        # JSON 파일로 저장
        with open(fname, 'w', encoding="utf-8") as f:
            json.dump(pos_tag, f, ensure_ascii=False, indent="\t")
            print("// 형태소 전처리 파일 생성 완료")
    return data

In [41]:
check_pos()

// 형태소 전처리 파일 존재


,comment,risk
0,"[접속/Noun, 하/Suffix, 자마자/Noun, 제/Modifier, 가가/N...",1
1,"[엘프/Noun, 궁수/Noun, 하는데/Verb, 4/Number, 렙때/Noun...",1
2,"[왜/Noun, 안/VerbPrefix, 깔리냐/Verb, ../Punctuation]",1
3,"[유/Noun, 져/Verb, 엿/Modifier, 먹임/Verb]",1
4,[도박장/Noun],1
...,...,...
46716,"[실망/Noun, 시키지/Verb, 않았으면/Verb, .../Punctuation...",0
46717,"[거래소/Noun, 굿/Noun]",0
46718,"[굿/Noun, ^^/Punctuation]",0
46719,"[최고/Noun, 의/Josa, 게임/Noun]",0


In [42]:
def word_select(data, num=10000):
    fname = "word_index.json"
    tokens = [token for row in data['comment'] for token in row]
    print("// 고유 단어 :", len(set(tokens)))
    text = nltk.Text(tokens, name='NMSC')#토큰이 몇개 있는지 확인
    selected_words = [t[0] for t in text.vocab().most_common(num)]
    with open(fname, 'w', encoding="utf-8") as f:
        json.dump(selected_words, f, ensure_ascii=False, indent="\t")
        print("// 단어 인덱스 파일 생성 완료")
    return selected_words

In [43]:
word_select(check_pos())

// 형태소 전처리 파일 존재
// 고유 단어 : 46614
// 단어 인덱스 파일 생성 완료


['게임/Noun',
 '이/Josa',
 './Punctuation',
 '도/Josa',
 '?/Punctuation',
 '가/Josa',
 '에/Josa',
 '을/Josa',
 '../Punctuation',
 '접속/Noun',
 '은/Josa',
 '리니지/Noun',
 '들/Suffix',
 '.../Punctuation',
 '는/Josa',
 '돈/Noun',
 '안/VerbPrefix',
 '현질/Noun',
 '만/Josa',
 '를/Josa',
 '이/Determiner',
 '으로/Josa',
 '겜/Noun',
 '왜/Noun',
 '의/Josa',
 '로/Josa',
 '서버/Noun',
 '과금/Noun',
 '좀/Noun',
 '케릭/Noun',
 '요/Josa',
 '유저/Noun',
 '못/VerbPrefix',
 '다/Adverb',
 '하고/Josa',
 '하/Suffix',
 '너무/Adverb',
 '추억/Noun',
 '하는/Verb',
 '뭐/Noun',
 '거/Noun',
 '유도/Noun',
 '진짜/Noun',
 'ㅡㅡ/KoreanParticle',
 '입니다/Adjective',
 '최악/Noun',
 '그냥/Noun',
 '에서/Josa',
 '업데이트/Noun',
 '한/Determiner',
 '캐릭/Noun',
 '고/Josa',
 '계정/Noun',
 '사람/Noun',
 '다시/Noun',
 '캐릭터/Noun',
 '!/Punctuation',
 '생각/Noun',
 '1/Number',
 '말/Noun',
 '최고/Noun',
 'ㅋㅋ/KoreanParticle',
 '엔씨/Noun',
 '했는데/Verb',
 '생/Noun',
 '하면/Verb',
 '할/Verb',
 '굿/Noun',
 '이/Noun',
 '삭제/Noun',
 '계속/Noun',
 '게/Josa',
 ',/Punctuation',
 '개/Noun',
 '아니/Adjective',
 '시작/Noun',
 '아인/Noun',
 

In [44]:
def counter_vect(data, word_index):
    print("// 워드 인덱스 정리 & 데이터 float화 진행")
    def term_frequency(doc):
        return [doc.count(word) for word in word_index]
    pre_data = np.asarray([term_frequency(d) for d in list(data['comment'])]).astype('float32')
    pre_label = data['risk'].astype('float32')
    print("// 전처리 최종 완료")
    return (pre_data, pre_label)

In [45]:
def deep_learning(dataset):
    train_data, test_data, train_label, test_label = train_test_split(dataset[0], dataset[1])
    
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))#모델 추가및 재거

    model.compile(optimizer=optimizers.RMSprop(lr=0.001),##
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

    model.fit(train_data, train_label, epochs=100,batch_size=512)   
    print("// 평가 :", model.evaluate(test_data, test_label))
    print("// 모델 저장 완료") 
    model.save('model.test')
    return model

In [47]:
data = check_pos()
word_index = word_select(data)
dataset = counter_vect(data, word_index)

// 형태소 전처리 파일 존재
// 고유 단어 : 46614
// 단어 인덱스 파일 생성 완료
// 워드 인덱스 정리 & 데이터 float화 진행
// 전처리 최종 완료


In [48]:
model = deep_learning(dataset)

Epoch 1/100
35040/35040 [==============================] - 3s 88us/step - loss: 0.4608 - binary_accuracy: 0.8037
Epoch 2/100
35040/35040 [==============================] - 3s 78us/step - loss: 0.2887 - binary_accuracy: 0.8909
Epoch 3/100
35040/35040 [==============================] - 3s 80us/step - loss: 0.2341 - binary_accuracy: 0.9082
Epoch 4/100
35040/35040 [==============================] - 3s 78us/step - loss: 0.2032 - binary_accuracy: 0.9211
Epoch 5/100
35040/35040 [==============================] - 3s 78us/step - loss: 0.1793 - binary_accuracy: 0.9313
Epoch 6/100
35040/35040 [==============================] - 3s 80us/step - loss: 0.1594 - binary_accuracy: 0.9419
Epoch 7/100
35040/35040 [==============================] - 3s 80us/step - loss: 0.1420 - binary_accuracy: 0.9493: 1s - loss: 0.1352 - bin
Epoch 8/100
35040/35040 [==============================] - 3s 82us/step - loss: 0.1267 - binary_accuracy: 0.9551
Epoch 9/100
35040/35040 [==============================] - 3s 82us/step

35040/35040 [==============================] - 3s 77us/step - loss: 0.0251 - binary_accuracy: 0.9901
Epoch 72/100
35040/35040 [==============================] - 3s 83us/step - loss: 0.0246 - binary_accuracy: 0.9905
Epoch 73/100
35040/35040 [==============================] - 3s 77us/step - loss: 0.0249 - binary_accuracy: 0.9902
Epoch 74/100
35040/35040 [==============================] - 3s 80us/step - loss: 0.0248 - binary_accuracy: 0.9902
Epoch 75/100
35040/35040 [==============================] - 3s 79us/step - loss: 0.0246 - binary_accuracy: 0.9903
Epoch 76/100
35040/35040 [==============================] - 3s 79us/step - loss: 0.0245 - binary_accuracy: 0.9903
Epoch 77/100
35040/35040 [==============================] - 3s 77us/step - loss: 0.0247 - binary_accuracy: 0.9902
Epoch 78/100
35040/35040 [==============================] - 3s 80us/step - loss: 0.0247 - binary_accuracy: 0.9904
Epoch 79/100
35040/35040 [==============================] - 3s 77us/step - loss: 0.0245 - binary_accu

In [49]:
model = load_model("model.h5")

In [50]:
def predict_risk(doc, word_index, model):
    okt = Okt()
    def tokenize(doc):
        return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]
        # norm은 정규화(오타 및 띄어쓰기 교정), stem은 근어화
    token = tokenize(doc)
    data = np.expand_dims(np.asarray([token.count(word) for word in word_index]).astype('float32'), axis=0)
    score = float(model.predict(data))
    print("[{}]의 위험 지수는 {:.2f}% 입니다.\n".format(doc, score * 100))

In [51]:
predict_risk("요정 사기다", word_index, model)

[요정 사기다]의 위험 지수는 27.22% 입니다.



In [52]:
predict_risk("최소 8강12회정도는 만들어논 사람들은 어케함? 돌리면 마이너스 될거 같은데 ㅡㅡ",word_index, model)

[최소 8강12회정도는 만들어논 사람들은 어케함? 돌리면 마이너스 될거 같은데 ㅡㅡ]의 위험 지수는 100.00% 입니다.



In [53]:
predict_risk("신검 접속했는데세인트 카운터가 계속 터지더라 피가 안달아ㅋㅋㅋ 그래서 미친하고 오늘 접속했는데 평상시 보나 물약을 200개나 더 먹어 ㅅㅂ 이렇게 지들 멋대로 확률 올렸다 내렸다해도 되는거임??? 쓰레기식희들...",word_index, model)

[신검 접속했는데세인트 카운터가 계속 터지더라 피가 안달아ㅋㅋㅋ 그래서 미친하고 오늘 접속했는데 평상시 보나 물약을 200개나 더 먹어 ㅅㅂ 이렇게 지들 멋대로 확률 올렸다 내렸다해도 되는거임??? 쓰레기식희들...]의 위험 지수는 0.00% 입니다.



In [54]:
predict_risk("농사서버 기웃거리는 형들을 위한 희소식",word_index, model)

[농사서버 기웃거리는 형들을 위한 희소식]의 위험 지수는 0.00% 입니다.



In [55]:
#서버 홍보 이야기
predict_risk("농사꾼형들. 팩트만 알려줄께 아툰 그랜드 끝난거 알제? 아툰9섭은 라인이 다 이전가서 라인이없음.. 이번주 중립이 3성 각임.87렙 초반이 3별 거뜬함. 노 통제 서버임.농사꾼형들 초대합니다.ㅡ 아.. 감옥 서버라고 걱정하거나 ,왜 글 쓰냐고 말하는 형들에게..농사는 개조은데, 사람이 없어서 템거래가 ..그래서 난 라인들말구 중립 농사형들이 왔음 좋겠엉세상 어느 시골에 라인없고 노통제 서버 봤음??글구, 담 그랜드 알제? 어느감옥이든 티켓타고 그냥 나가는거.농사는 한철이야 형들.통제 없는 우리서버로와. 라인형들 말구 ㅡㅡ",word_index, model)

[농사꾼형들. 팩트만 알려줄께 아툰 그랜드 끝난거 알제? 아툰9섭은 라인이 다 이전가서 라인이없음.. 이번주 중립이 3성 각임.87렙 초반이 3별 거뜬함. 노 통제 서버임.농사꾼형들 초대합니다.ㅡ 아.. 감옥 서버라고 걱정하거나 ,왜 글 쓰냐고 말하는 형들에게..농사는 개조은데, 사람이 없어서 템거래가 ..그래서 난 라인들말구 중립 농사형들이 왔음 좋겠엉세상 어느 시골에 라인없고 노통제 서버 봤음??글구, 담 그랜드 알제? 어느감옥이든 티켓타고 그냥 나가는거.농사는 한철이야 형들.통제 없는 우리서버로와. 라인형들 말구 ㅡㅡ]의 위험 지수는 100.00% 입니다.



In [56]:
#버그 관련 이야기
predict_risk("대박 유튜브 대만인? 자기서버? 방송하는거 보는데 몬스터 체력이보임 ㅋㅋㅋ공격하면 피도 닳는거도 보임 대박 뭔 버그 프로그램을 쓰는거야 ㅋㅋ 이런거 처음봄pvp할때도 상대방 체력 보이는거 아님? ㅋㅋ",word_index, model)

[대박 유튜브 대만인? 자기서버? 방송하는거 보는데 몬스터 체력이보임 ㅋㅋㅋ공격하면 피도 닳는거도 보임 대박 뭔 버그 프로그램을 쓰는거야 ㅋㅋ 이런거 처음봄pvp할때도 상대방 체력 보이는거 아님? ㅋㅋ]의 위험 지수는 100.00% 입니다.



In [57]:
#안좋은거 건졌다는 이야기
predict_risk("8면갑 8무갑(각인) 7보망 날리고 8완부(각인) 건졌는데 이득임? 근데 각인임 시발",word_index, model)

[8면갑 8무갑(각인) 7보망 날리고 8완부(각인) 건졌는데 이득임? 근데 각인임 시발]의 위험 지수는 100.00% 입니다.



In [58]:
predict_risk("택진",word_index, model)

[택진]의 위험 지수는 50.66% 입니다.



In [59]:
predict_risk("영자야 겜이 3년째 가고있으면 내가봤을때 교환기능은 한 3년뒤에나 나올거같고 그러면 캐릭터 눌렀을때 우측에 있는 '교환'이라는 문구는  그냥 없애주던가 ^ㅗ^ 아니면 땕쉙읉떩뿕 이딴거 신고좀 편하게 하게 되지도 않는 교환기능 빼고 신고하기로 바꿔주던가 좀 일 좀 해 c8 좀 패키지나 처 만들생각 하지말고  진또배기 유저들 잡을 생각해야지 매크로 살려둬서 니네들이 득 본게 피니지때도 그렇고 있긴허냐?",word_index, model)

[영자야 겜이 3년째 가고있으면 내가봤을때 교환기능은 한 3년뒤에나 나올거같고 그러면 캐릭터 눌렀을때 우측에 있는 '교환'이라는 문구는  그냥 없애주던가 ^ㅗ^ 아니면 땕쉙읉떩뿕 이딴거 신고좀 편하게 하게 되지도 않는 교환기능 빼고 신고하기로 바꿔주던가 좀 일 좀 해 c8 좀 패키지나 처 만들생각 하지말고  진또배기 유저들 잡을 생각해야지 매크로 살려둬서 니네들이 득 본게 피니지때도 그렇고 있긴허냐?]의 위험 지수는 100.00% 입니다.



In [60]:
predict_risk("오늘 컬렉도전할껀데 러쉬해보신분들 뭐가 더 띄우기 힘든지 알려주시면 감사하겠습니다.. 느낌상 8갑옷이 더 힘들거같긴한데..",word_index, model)

[오늘 컬렉도전할껀데 러쉬해보신분들 뭐가 더 띄우기 힘든지 알려주시면 감사하겠습니다.. 느낌상 8갑옷이 더 힘들거같긴한데..]의 위험 지수는 100.00% 입니다.



In [61]:
predict_risk("형님들 걍 이반 시세 얼마임?",word_index, model)

[형님들 걍 이반 시세 얼마임?]의 위험 지수는 100.00% 입니다.



In [62]:
predict_risk("택진이 돈에 환장했네 ",word_index, model)

[택진이 돈에 환장했네 ]의 위험 지수는 37.65% 입니다.



In [63]:
predict_risk('''신성검사 세인트 카운터 발동확률 급상승한듯 합니다.
머리위에 방패가 계속 뜨면서 힐이 계속 들어가네요
확률 조정한답시고 묻지마 상향 했는데 수치를 잘못 건들인듯합니다 ㅋㅋㅋㅋㅋㅋㅋ
빠른 긴급점검 부탁드립니다''',word_index, model)

[신성검사 세인트 카운터 발동확률 급상승한듯 합니다.
머리위에 방패가 계속 뜨면서 힐이 계속 들어가네요
확률 조정한답시고 묻지마 상향 했는데 수치를 잘못 건들인듯합니다 ㅋㅋㅋㅋㅋㅋㅋ
빠른 긴급점검 부탁드립니다]의 위험 지수는 0.00% 입니다.



In [64]:
predict_risk('''신성검사 세인트 카운터 발동확률 급상승한듯 합니다.
머리위에 방패가 계속 뜨면서 힐이 계속 들어가네요
확률 조정한답시고 묻지마 상향 했는데 수치를 잘못 건들인듯합니다 ㅋㅋㅋㅋㅋㅋㅋ
빠른 긴급점검 부탁드립니다''',word_index, model)

[신성검사 세인트 카운터 발동확률 급상승한듯 합니다.
머리위에 방패가 계속 뜨면서 힐이 계속 들어가네요
확률 조정한답시고 묻지마 상향 했는데 수치를 잘못 건들인듯합니다 ㅋㅋㅋㅋㅋㅋㅋ
빠른 긴급점검 부탁드립니다]의 위험 지수는 0.00% 입니다.



In [65]:
predict_risk('''크리스마스 방어구에서 첫타에 나왔어요.....
드다상자에서 반역자의방패 나오고...이번엔.....신완각을 주네요.....
이러니...리니지를 벗어나지를 못하지 ㅠㅠ정말.....''',word_index, model)

[크리스마스 방어구에서 첫타에 나왔어요.....
드다상자에서 반역자의방패 나오고...이번엔.....신완각을 주네요.....
이러니...리니지를 벗어나지를 못하지 ㅠㅠ정말.....]의 위험 지수는 100.00% 입니다.



In [66]:
predict_risk('''뭐 리니지m 하시는 분들한테는 그저익숙한 게임속 풍경이긴 하겟지요.
장관이긴 합니다. 월보시간이면 아덴월드 마을에 복귀해서우수수 날아가는 장면들로서스섬도 뭐 장난아니고기란던전이야 작업장의 천국이 된지는 오래고요.
어차피 매크로는 치면 날라가는데 그나마중립유저들 가끔 희귀라도 빼먹는 파우스트 패치 ㅎㅎ''',word_index, model)

[뭐 리니지m 하시는 분들한테는 그저익숙한 게임속 풍경이긴 하겟지요.
장관이긴 합니다. 월보시간이면 아덴월드 마을에 복귀해서우수수 날아가는 장면들로서스섬도 뭐 장난아니고기란던전이야 작업장의 천국이 된지는 오래고요.
어차피 매크로는 치면 날라가는데 그나마중립유저들 가끔 희귀라도 빼먹는 파우스트 패치 ㅎㅎ]의 위험 지수는 100.00% 입니다.



In [67]:
predict_risk(''' 일기장 12개 모았는데 첫완부로 뜨네요.
나머지 일기장은 일단 킵해야겠어요.
신검인데 신발졸업해서 기쁩니다.''',word_index, model)

[ 일기장 12개 모았는데 첫완부로 뜨네요.
나머지 일기장은 일단 킵해야겠어요.
신검인데 신발졸업해서 기쁩니다.]의 위험 지수는 100.00% 입니다.

